In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
txt = pd.read_excel('数据比较好的.xlsx')
Y = txt['标签']
jieba.load_userdict(Y)
stopwords = [line.strip() for line in open('../input/stopword.txt', 'r', encoding='utf-8').readlines()] 
list_des = txt['商品描述'].tolist()
X = []
with open('分词拿去做word2vect.txt','w',encoding = 'utf8') as file:
    for i in range(len(list_des)):
        des = list_des[i].replace(' ','')
        seg_list = jieba.lcut(des)
        word_list = [] 
        for seg in seg_list:
            if seg not in stopwords:
                word_list.append(seg)
        X.append((' ').join(word_list))
        file.write((' ').join(word_list) +'\n')
txt['商品描述'] = pd.Series(X) 

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hebo\AppData\Local\Temp\jieba.cache
Loading model cost 0.797 seconds.
Prefix dict has been built succesfully.


In [2]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [3]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
        continue
    elif dic[i]>=30:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)

frame = frame.reset_index(drop=True)
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

数据量：40387 分类数目：511


In [4]:
vectorizer = CountVectorizer()
fre_matrix = vectorizer.fit_transform(frame['商品描述'])
feature_names  = vectorizer.get_feature_names()
word_index_dic = vectorizer.vocabulary_
length = (fre_matrix.shape)[1]
newd = {v:k for k,v in word_index_dic.items()}
bag_words = []
tfidf = TfidfTransformer()
tfidf = tfidf.fit_transform(fre_matrix)

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.1)

In [22]:
def LR(X,Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.1)
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, Y_train)# coding:utf-8
    y_pred = classifier.predict(X_test)
    print(np.mean(y_pred == Y_test))
    return (np.mean(y_pred == Y_test))

In [23]:
accuracy = LR(fre_matrix,frame['标签'])

0.8346534653465346


In [24]:
accuracy = LR(tfidf,frame['标签'])

0.8116336633663367


In [7]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
clf = svm.SVC(kernel = 'linear')
#x_chart = StandardScaler(with_mean=False)
#x_scaler = x_chart.fit_transform(tfidf)
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.2)
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
np.mean(y_pre == Y_test)

0.8514483783114632

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
mnb = MultinomialNB()   # 使用默认配置初始化朴素贝叶斯
mnb.fit(X_train,Y_train)    # 利用训练数据对模型参数进行估计
y_predict = mnb.predict(X_test)     # 对参数进行预测
print('The Accuracy of Naive Bayes Classifier is:', mnb.score(X_test,Y_test))
print(classification_report(Y_test, y_predict, target_names = Y_test.values))

The Accuracy of Naive Bayes Classifier is: 0.6391089108910891
             precision    recall  f1-score   support

        普洱茶       0.65      0.74      0.69        27
         支架       0.00      0.00      0.00         6
         帐篷       0.00      0.00      0.00         2
         手环       0.00      0.00      0.00         4
        保护套       0.00      0.00      0.00         4
         床单       0.50      0.33      0.40         3
        保护套       0.67      0.80      0.73        10
         大衣       0.61      0.61      0.61        18
         内衣       0.83      0.71      0.77         7
         帽子       1.00      0.50      0.67         4
        假睫毛       1.00      0.50      0.67         2
        单肩包       0.00      0.00      0.00         3
         墨水       0.00      0.00      0.00         2
        单肩包       1.00      0.71      0.83         7
         贴纸       0.00      0.00      0.00         3
         发夹       0.80      0.67      0.73         6
         面膜       0.00      0.00    

c:\python3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 506, does not match size of target_names, 4040
  .format(len(labels), len(target_names))
c:\python3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
from sklearn import tree 
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
np.mean(y_pre == Y_test)

0.8133663366336633

In [29]:
from sklearn.ensemble import RandomForestClassifier
clf  = RandomForestClassifier()
clf.fit(X_train,Y_train)
y_pre = clf.predict(X_test)
np.mean(y_pre == Y_test)

0.7856435643564357

In [10]:
n = 0
for label in Counter(frame['标签']).keys():
    if n % 100 == 0:
        print(n)
    choose = frame.loc[frame['标签'] == label].index
    A = fre_matrix[choose].sum(axis=0)
    C = np.zeros((1,length),np.int64)
    index_list = [] 
    for i in range(length):
        if A[0,i] != 0:
            index_list.append(i)
            C[0,i] = len(choose) - A[0,i]
    choose_1 = frame.loc[frame['标签'] != label].index
    B_ = np.zeros((1,length),np.int64)
    for index in choose_1:
        B_ = fre_matrix[index] + B_
    B = np.zeros((1,length),np.int64)
    for index in index_list:
        B[0,index] = B_[0,index]
    D = np.zeros((1,length),np.int64)
    for i in range(length):
        if B[0,i] != 0:
            D[0,i] = len(choose_1) - B[0,i]
    AD_BC_2 = (np.array(A)*np.array(D) - np.array(B)*np.array(C))**2
    chi2 =  AD_BC_2  / ((np.array(A) + np.array(B))*(np.array(C)+ np.array(D)))
    hope_index = {}
    for i in range(length):
        if isnan(chi2[0,i]) == False:
            hope_index[i] = chi2[0,i]
    li = sorted(hope_index.items(),key = lambda x:x[1],reverse = True)
    #for i in range(2):
        #bag_words.append(newd[li[i][0]])
    bag_words.append(newd[li[0][0]])
    n = n + 1

0


c:\python3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [11]:
import copy
good_words = list(set(bag_words))
def _chi2(matrix,weight):
    matrix_1 = copy.deepcopy(matrix)
    for row in range(matrix_1.shape[0]):
        for word in frame['商品描述'][row].split():
            if word in good_words:
                col = word_index_dic[word]
                matrix_1[row,col] += weight
    return matrix_1
X_matrix = _chi2(fre_matrix,1)
accuracy = LR(X_matrix,frame['标签'])

0.7858676975945017


In [12]:
X_matrix = _chi2(fre_matrix,0.5)
accuracy = LR(X_matrix,frame['标签'])

0.7712628865979382


In [13]:
X_matrix = _chi2(fre_matrix,1.5)
accuracy = LR(X_matrix,frame['标签'])

0.7875859106529209


In [16]:
X_matrix = _chi2(tfidf,3)
accuracy = LR(X_matrix,frame['标签'])

0.791881443298969
